In [1]:
# https://github.com/rapidsai/cuspatial/issues/886
import geopandas as gpd
import cuspatial
host_dataframe = gpd.read_file(
    gpd.datasets.get_path("naturalearth_lowres")
)
dataframe = cuspatial.from_geopandas(host_dataframe)
dataframe

In [12]:
# https://github.com/rapidsai/cuspatial/issues/887
import cupy as cp
import cudf
long = cudf.Series(cp.arange(20000000), dtype="float64")
series = cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(long._column))
series

0                        POINT (0.000 1.000)
1                        POINT (2.000 3.000)
2                        POINT (4.000 5.000)
3                        POINT (6.000 7.000)
4                        POINT (8.000 9.000)
                         ...                
9999995    POINT (19999990.000 19999991.000)
9999996    POINT (19999992.000 19999993.000)
9999997    POINT (19999994.000 19999995.000)
9999998    POINT (19999996.000 19999997.000)
9999999    POINT (19999998.000 19999999.000)
Length: 10000000, dtype: geometry

In [13]:
# https://github.com/rapidsai/cuspatial/issues/888
import cupy as cp
import cudf
long = cudf.Series(cp.arange(20))
series = cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(long._column))
series

0    POINT (0.00000 0.00000)
1    POINT (0.00000 0.00000)
2    POINT (0.00000 0.00000)
3    POINT (0.00000 0.00000)
4    POINT (0.00000 0.00000)
5    POINT (0.00000 0.00000)
6    POINT (0.00000 0.00000)
7    POINT (0.00000 0.00000)
8    POINT (0.00000 0.00000)
9    POINT (0.00000 0.00000)
dtype: geometry

In [16]:
# https://github.com/rapidsai/cuspatial/issues/889
import cuspatial
from shapely.geometry import MultiPolygon, Polygon, Point
def get_polygon():
    return Polygon([[0, 0], [1, 0], [1, 1]])
series = cuspatial.GeoSeries([MultiPolygon([get_polygon() for i in range(32)])])
series.contains_properly(cuspatial.GeoSeries([Point(0, 0)]))

RuntimeError: cuSpatial failure at: /home/tcomer/mnt/NVIDIA/rapids-docker/cuspatial/cpp/src/spatial/point_in_polygon.cu:140: Number of polygons cannot exceed 31

In [5]:
# https://github.com/rapidsai/cuspatial/issues/890
import cuspatial
import cudf
import geopandas
import cupy as cp
host_zones = geopandas.read_file('taxi_zones.zip')
host_lonlat = host_zones.to_crs(epsg=4326)
zones = cuspatial.from_geopandas(host_lonlat)
taxi2015 = cudf.read_csv('taxi2015.csv')
def quadtree(polygons, points):
    poly_points_x = polygons.polygons.x
    poly_points_y = polygons.polygons.y
    poly_offsets = polygons.polygons.part_offset
    poly_ring_offsets = polygons.polygons.ring_offset
    test_points_x = points.points.x
    test_points_y = points.points.y
    scale = 1
    max_depth = 15
    min_size = 125
    x_max = poly_points_x.max()
    x_min = poly_points_x.min()
    y_max = poly_points_y.max()
    y_min = poly_points_y.min()
    point_indices, quadtree = cuspatial.quadtree_on_points(
        test_points_x,
        test_points_y,
        x_min,
        x_max,
        y_min,
        y_max,
        scale,
        max_depth,
        min_size,
    )
    poly_bboxes = cuspatial.polygon_bounding_boxes(
        poly_offsets, poly_ring_offsets, poly_points_x, poly_points_y
    )
    intersections = cuspatial.join_quadtree_and_bounding_boxes(
        quadtree, poly_bboxes, x_min, x_max, y_min, y_max, scale, max_depth
    )
    polygons_and_points = cuspatial.quadtree_point_in_polygon(
        intersections,
        quadtree,
        point_indices,
        test_points_x,
        test_points_y,
        poly_offsets,
        poly_ring_offsets,
        poly_points_x,
        poly_points_y,
    )
    return polygons_and_points
def make_geoseries_from_lon_lat(lon, lat):
    # Scatter the two columns into one column
    assert len(lon) == len(lat)
    xy = cudf.Series(cp.zeros(len(lon) * 2))
    xy[::2] = lon
    xy[1::2] = lat

    return cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(xy._column))
dropoffs = make_geoseries_from_lon_lat(
    taxi2015['dropoff_longitude'],
    taxi2015['dropoff_latitude']
)
#print(quadtree(zones['geometry'], dropoffs))
#print(quadtree(zones['geometry'].iloc[0:120], dropoffs))
print(quadtree(zones['geometry'].iloc[0:len(zones)-10], dropoffs))

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/tcomer/mnt/NVIDIA/rapids-docker/rmm/include/rmm/mr/device/cuda_memory_resource.hpp:70: cudaErrorMemoryAllocation out of memory

In [27]:
# https://github.com/rapidsai/cuspatial/issues/891
import geopandas as gpd
import cuspatial
dataframe = cuspatial.GeoDataFrame(gpd.read_file(
    gpd.datasets.get_path("naturalearth_lowres")
))
dataframe['geometry'].polygons.point_indices()

RuntimeError: CUDF failure at: /home/tcomer/mnt/NVIDIA/rapids-docker/cudf/cpp/src/filling/repeat.cu:109: in and count must have equal size

In [2]:
# https://github.com/cuspatial/issues/896
import geopandas as gpd
import cuspatial
df = cuspatial.GeoDataFrame(
    gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
)
df['geometry'].contains_properly(df['geometry'])

RuntimeError: CUDF failure at: /home/tcomer/mnt/NVIDIA/rapids-docker/cudf/cpp/src/filling/repeat.cu:109: in and count must have equal size

In [3]:
# https://github.com/rapidsai/cuspatial/issues/897
import cupy as cp
import cudf
import cuspatial
long = cudf.Series(cp.arange(4), dtype="float64")
series = cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(long._column))
series.multipoints.xy

AttributeError: 'NumericalColumn' object has no attribute 'leaves'

In [10]:
# https://github.com/rapidsai/cuspatial/issues/895
# quadtree degenerate case
import cuspatial
import cudf
import geopandas
import cupy as cp
from shapely.geometry import Polygon, Point
def quadtree(polygons, points):
    poly_points_x = polygons.polygons.x
    poly_points_y = polygons.polygons.y
    poly_offsets = polygons.polygons.part_offset
    poly_ring_offsets = polygons.polygons.ring_offset
    test_points_x = points.points.x
    test_points_y = points.points.y
    scale = 1
    max_depth = 15
    min_size = 125
    x_max = poly_points_x.max()
    x_min = poly_points_x.min()
    y_max = poly_points_y.max()
    y_min = poly_points_y.min()
    point_indices, quadtree = cuspatial.quadtree_on_points(
        test_points_x,
        test_points_y,
        x_min,
        x_max,
        y_min,
        y_max,
        scale,
        max_depth,
        min_size,
    )
    poly_bboxes = cuspatial.polygon_bounding_boxes(
        poly_offsets, poly_ring_offsets, poly_points_x, poly_points_y
    )
    intersections = cuspatial.join_quadtree_and_bounding_boxes(
        quadtree, poly_bboxes, x_min, x_max, y_min, y_max, scale, max_depth
    )
    print(poly_offsets)
    print(poly_ring_offsets)
    polygons_and_points = cuspatial.quadtree_point_in_polygon(
        intersections,
        quadtree,
        point_indices,
        test_points_x,
        test_points_y,
        poly_offsets,
        poly_ring_offsets,
        poly_points_x,
        poly_points_y,
    )
    return polygons_and_points
points = cuspatial.GeoSeries([Point(0.5, 0.5), Point(0.5, 0.5)])
polygons = cuspatial.GeoSeries([
    Polygon([[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]])
])
quadtree(polygons, points)

[0 1]
[0 5]


RuntimeError: cuSpatial failure at: /home/tcomer/mnt/NVIDIA/rapids-docker/cuspatial/cpp/src/join/quadtree_point_in_polygon.cu:271: all rings must have at least 3 vertices